# Wires game(s)

This noteboook will generate a contitional wires game with the text instructions and arduino code

In [2]:
from copy import deepcopy
import random
from typing import Any, Dict, List

In [3]:
states_dict = {
    'open': {
        'sensor_val': 0,
        'manipulation': 'pull out the {{wire}} wire'
    },
    'low': {
        'sensor_val': 100,
        'manipulation': 'plug the {{wire}} wire into the bottom rail'
    },
    'high': {
        'sensor_val': 500,
        'manipulation': 'plug the {{wire}} wire into the top rail'
    }
}

states = list(states_dict.keys())

In [4]:
wires = [
    'red',
    'yellow',
    'orange',
    'white',
    'blue',
    'green',
    # 'black'
]

wires_dict = {wires[i]: f'A{i}' for i in range(len(wires))}
wires_dict

{'red': 'A0',
 'yellow': 'A1',
 'orange': 'A2',
 'white': 'A3',
 'blue': 'A4',
 'green': 'A5'}

In [5]:
combis_list = []

for s in states:
    for w in wires:
        combis_list.append([s, w])
        
combis_list

[['open', 'red'],
 ['open', 'yellow'],
 ['open', 'orange'],
 ['open', 'white'],
 ['open', 'blue'],
 ['open', 'green'],
 ['low', 'red'],
 ['low', 'yellow'],
 ['low', 'orange'],
 ['low', 'white'],
 ['low', 'blue'],
 ['low', 'green'],
 ['high', 'red'],
 ['high', 'yellow'],
 ['high', 'orange'],
 ['high', 'white'],
 ['high', 'blue'],
 ['high', 'green']]

In [6]:
lights = [
    'top',
    'middle',
    'bottom'
]

light_states = [
    'on',
    'off'
]

In [7]:
code_template = """

/*
  Auto-generated wiring program template for wires game.

  Reads against 3 states:
   - High rail connected
   - Low rail connected
   - Disconnected

  High rail sould be connected to 3V3 terminal.
  Low rail should be connected by 10K to 3V3 and 10K to GND (sits between two 10K resistors).
  All input wires should have 100K pull-down to GND, then conncted to inputs specified below. 
*/


{{wire_pin_declarations}}

const int low_thresh = 100;
const int up_thresh = 500;

bool wire_change = false;
int progress = 0;


int mapPinValue(int input) {
  // Maps the input value against uneven thresholds for 3-level mapping
  // open = 0, low = 1, high = 2
  int output = 0;
  if (input > up_thresh) {
    output = 2;
  }
  else {
    if (input > low_thresh) {
      output = 1;
    }
  }
  return output;
}


int get_curr_pin_state(int pin) {
  // Gets the current mapped state of a given pin
  int val_in = analogRead(pin);
  val_in = mapPinValue(val_in);
  return val_in;
}


void safe() {
  // actions upon game won
  ToDo: <<<<<<<<<<
}


void lose () {
  // actions upon game lose
  ToDo: <<<<<<<<<<
}


{{game_functions}}


void setup() {
{{setup_code}}
}


void loop() {
  //game loop
{{loop_code}}
}

"""

# Game 1: Sequence game

Pull out one of the wires <br>
 - LEDs light in random configuration
 - Pull out white and blue wires
 - Plug green wire to bottom rail
 - Plug the blue wire into the bottom rail
 - Pull out the yellow wire


In [9]:
sequence_game_code_functions = """

int check_progress() {
  switch (progress) {
    {{case_statements}}
    default:
      _states;
  }
  
}
"""

sequence_game_setup = """
  // set the current wire conditions (and allowed opens to starting opens)
  for (int i=0; i<{{num_wires}}; i++) {
    int curr_pin_state = get_curr_pin_state(wires[i]);
    curr_pin_states[i] = curr_pin_state;
    if (curr_pin_state == 0) {
      allowed_opens[i] = 1;
    }

  }
  
  // set the sequence indicator LEDs
  ToDo: <<<<<<<<<<

"""

sequence_game_loop = """
  // run the game logic
  wire_change = false;
  // check for wire change
  for (int i=0; i<6; i++) {
    int curr_pin_state = get_curr_pin_state(wires[i])
    int prev_pin_state = curr_pin_states[i]
    if (curr_pin_state != prev_pin_state) {
      wire_change = true;
    }
  }

  if (wire_change == true) {
    check_progress()
  }

  if (progress > 6) {
    safe();
  }
  else {
    if (progress < 0) {
      lose();
    }
  }

"""

In [10]:
used_start_light_states = []

class sequence_object:
    """
    Stores an instance of a sub-game (sequence isntructions and corresponding hardware attributes) 
    """
    def __init__(self,
                 code_template: str,
                 num_instructions: int = 5) -> None:
        """
        :param num_instructions: the length of the sequence to be carried out in instruction steps
        """
        self.num_instructions: int = num_instructions
        self.instructions_list = []
        self.instruction_states = []
        
        self.start_lights = {}
        self.start_lights = {}
        
        self._required_wire_states = {}
        
        self.code_template: str = code_template
    
    
    def generate_start_lights(self) -> None:
        """
        Function produces the indicating start light configuration
        """
        unused_start_lights = False
        while not unused_start_lights:
            self.start_lights = [random.choice(light_states) for light in lights]
            if self.start_lights not in used_start_light_states:
                used_start_light_states.append(self.start_lights)
                unused_start_lights = True
    
    
    def generate_start_light_text(self) -> str:
        """
        Function produces the indicating start light configuration
        """
        if not self.start_lights:
            self.generate_start_lights()
        return "\n".join(self.start_lights)
    
    
    def generate_step(self) -> None:
        """
        Genererates an indivudal step instructions and states
        """
        num_step_wires = random.randint(1, 2)
        step_wires = {}
        wire_instructions = []
        for i in range(num_step_wires):
            step_wires[random.choice([wire for wire in wires if wire not in step_wires])] = {}
            
        for wire in step_wires:
            
            wire_state = random.choice([state for state in states_dict
                                        if state != self._required_wire_states.get(wire, None)])
            step_wires[wire] = wire_state
            self._required_wire_states[wire] = wire_state
            
            wire_instruction = states_dict[wire_state]['manipulation'].replace('{{wire}}', wire)
            wire_instructions.append(wire_instruction)
        
        step_instruction = " and ".join(wire_instructions)
        step_instruction_formatted = f'P{step_instruction[1:]}.'
        
        self.instructions_list.append(step_instruction_formatted)
        self.instruction_states.append(deepcopy(self._required_wire_states))
            
        # print(step_wires)
        # print(wire_instructions)
        # print('-------')
        # print(self._required_wire_states)
        # print(self.instructions_list)
        # print('')

        
    def generate_sequence(self) -> None:
        """
        Recursively generates the required number of instructions
        """
        for i in range(self.num_instructions):
            self.generate_step()
    
    
     # --------------------------------
        # Code genetarors
     # --------------------------------
    
    def generate_pin_declarations(self) -> None:
        """
        Auto-generates pin declarations for arduino code
        """
        # declare individual pins
        pin_declarations = []
        for wire, pin in wires_dict.items():
            pin_declarations.append(f"const int {wire}_pin = {pin};")
        
        # declare pin array
        pin_array_str = ', '.join([f'{wire}_pin' for wire in wires])
        pin_declarations.append(f"\nconst int wires[{len(wires)}] = [{pin_array_str}];")
        
        # declare pin states
        pin_states_str = ', '.join(['0'] * len(wires))
        pin_declarations.append(f"\nint curr_pin_states[{len(wires)}] = [{pin_states_str}];")
        pin_declarations.append(f"int allowed_opens[{len(wires)}] = [{pin_states_str}];")
        
        pin_dec_str = "\n".join(pin_declarations)
        self.code_template = self.code_template.replace('{{wire_pin_declarations}}', pin_dec_str)
        
    
    def generate_game_code(self) -> None:
        """
        Code generation function for producing the complete arduino code
        """
        self.generate_pin_declarations()
        self.code_template = self.code_template.replace('{{game_functions}}', sequence_game_code_functions)
        self.code_template = self.code_template.replace('{{setup_code}}', sequence_game_setup)
        self.code_template = self.code_template.replace('{{loop_code}}', sequence_game_loop)
        
        # update any for-loop lengths dependent upon number of wires
        self.code_template = self.code_template.replace('{{num_wires}}', str(len(wires)))
        print(self.code_template)
        


In [11]:
test_so = sequence_object(code_template)

In [12]:
print(test_so.generate_start_light_text())

off
on
off


In [13]:
test_so.generate_sequence()

In [14]:
test_so.instructions_list

['Plug the orange wire into the top rail and pull out the green wire.',
 'Pull out the yellow wire and pull out the white wire.',
 'Plug the white wire into the bottom rail and plug the blue wire into the bottom rail.',
 'Plug the yellow wire into the top rail and pull out the red wire.',
 'Pull out the orange wire and pull out the yellow wire.']

In [15]:
test_so.instruction_states

[{'orange': 'high', 'green': 'open'},
 {'orange': 'high', 'green': 'open', 'yellow': 'open', 'white': 'open'},
 {'orange': 'high',
  'green': 'open',
  'yellow': 'open',
  'white': 'low',
  'blue': 'low'},
 {'orange': 'high',
  'green': 'open',
  'yellow': 'high',
  'white': 'low',
  'blue': 'low',
  'red': 'open'},
 {'orange': 'open',
  'green': 'open',
  'yellow': 'open',
  'white': 'low',
  'blue': 'low',
  'red': 'open'}]

---

ToDo:
 - 
 - build the code generator

In [18]:
test_so.generate_game_code()



/*
  Auto-generated wiring program template for wires game.

  Reads against 3 states:
   - High rail connected
   - Low rail connected
   - Disconnected

  High rail sould be connected to 3V3 terminal.
  Low rail should be connected by 10K to 3V3 and 10K to GND (sits between two 10K resistors).
  All input wires should have 100K pull-down to GND, then conncted to inputs specified below. 
*/


const int red_pin = A0;
const int yellow_pin = A1;
const int orange_pin = A2;
const int white_pin = A3;
const int blue_pin = A4;
const int green_pin = A5;

const int wires[6] = [red_pin, yellow_pin, orange_pin, white_pin, blue_pin, green_pin];

int curr_pin_states[6] = [0, 0, 0, 0, 0, 0];
int allowed_opens[6] = [0, 0, 0, 0, 0, 0];

const int low_thresh = 100;
const int up_thresh = 500;

bool wire_change = false;
int progress = 0;


int mapPinValue(int input) {
  // Maps the input value against uneven thresholds for 3-level mapping
  // open = 0, low = 1, high = 2
  int output = 0;
  if (input >